IMDB-introduction (자연어 처리를 위해 사용)

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

tf.random.set_seed(777)
print(tf.__version__)

2.13.0


In [2]:
imdb = keras.datasets.imdb #총 50000개의 영화 리뷰를 가져와서 긍정 부정을 판단함
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)
print("Training entries: {}, labels: {}".format(len(train_data), len(train_labels)))
print(train_data[0])

17464789/17464789 [==============================] - 3s 0us/step
Training entries: 25000, labels: 25000
[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 447

In [3]:
# A dictionary mapping words to an integer index
word_index = imdb.get_word_index()

# The first indices are reserved
word_index = {k:(v+3) for k,v in word_index.items()}
word_index["<PAD>"] = 0             #자연어의 경우에는 전처리 과정이 중요한데 공백에 대한 값을 0, 시작할때 1, 등으로 표현할 수 있다.
word_index["<START>"] = 1
word_index["<UNK>"] = 2  # unknown
word_index["<UNUSED>"] = 3

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

decode_review(train_data[4])

1641221/1641221 [==============================] - 0s 0us/step


"<START> worst mistake of my life br br i picked this movie up at target for 5 because i figured hey it's sandler i can get some cheap laughs i was wrong completely wrong mid way through the film all three of my friends were asleep and i was still suffering worst plot worst script worst movie i have ever seen i wanted to hit my head up against a wall for an hour then i'd stop and you know why because it felt damn good upon bashing my head in i stuck that damn movie in the <UNK> and watched it burn and that felt better than anything else i've ever done it took american psycho army of darkness and kill bill just to get over that crap i hate you sandler for actually going through with this and ruining a whole day of my life"

In [12]:
print(train_labels[4])

0


In [5]:
train_data = keras.preprocessing.sequence.pad_sequences(train_data,
                                                        value=word_index["<PAD>"],
                                                        padding='post',
                                                        maxlen=256)

test_data = keras.preprocessing.sequence.pad_sequences(test_data,
                                                       value=word_index["<PAD>"],
                                                       padding='post',
                                                       maxlen=256)

print(len(train_data[0]), len(test_data[0]))
print(train_data[0])

256 256
[   1   14   22   16   43  530  973 1622 1385   65  458 4468   66 3941
    4  173   36  256    5   25  100   43  838  112   50  670    2    9
   35  480  284    5  150    4  172  112  167    2  336  385   39    4
  172 4536 1111   17  546   38   13  447    4  192   50   16    6  147
 2025   19   14   22    4 1920 4613  469    4   22   71   87   12   16
   43  530   38   76   15   13 1247    4   22   17  515   17   12   16
  626   18    2    5   62  386   12    8  316    8  106    5    4 2223
 5244   16  480   66 3785   33    4  130   12   16   38  619    5   25
  124   51   36  135   48   25 1415   33    6   22   12  215   28   77
   52    5   14  407   16   82    2    8    4  107  117 5952   15  256
    4    2    7 3766    5  723   36   71   43  530  476   26  400  317
   46    7    4    2 1029   13  104   88    4  381   15  297   98   32
 2071   56   26  141    6  194 7486   18    4  226   22   21  134  476
   26  480    5  144   30 5535   18   51   36   28  224   92   25  10

동일길이인 256으로 만주처주기 위하여 뒤에 여백은 0으로 맞춰줌

In [6]:
# input shape is the vocabulary count used for the movie reviews (10,000 words)
vocab_size = 10000 #10000가지의 단어 구별 가능

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation=tf.nn.relu))
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 16)          160000    
                                                                 
 global_average_pooling1d (  (None, 16)                0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 160289 (626.13 KB)
Trainable params: 160289 (626.13 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


model.add(keras.layers.Embedding(vocab_size, 16)) : Embedding layer에서 vocab_size= input _dim, output_dim = 16

In [7]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [8]:
x_val = train_data[:10000]               #10000을 기준으로 학습과 평가 수행
partial_x_train = train_data[10000:]

y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=40,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

Epoch 1/40
30/30 [==============================] - 3s 46ms/step - loss: 0.6922 - accuracy: 0.5497 - val_loss: 0.6908 - val_accuracy: 0.4950
Epoch 2/40
30/30 [==============================] - 1s 42ms/step - loss: 0.6870 - accuracy: 0.5716 - val_loss: 0.6833 - val_accuracy: 0.6950
Epoch 3/40
30/30 [==============================] - 1s 32ms/step - loss: 0.6747 - accuracy: 0.7018 - val_loss: 0.6672 - val_accuracy: 0.7395
Epoch 4/40
30/30 [==============================] - 1s 43ms/step - loss: 0.6515 - accuracy: 0.7553 - val_loss: 0.6405 - val_accuracy: 0.7466
Epoch 5/40
30/30 [==============================] - 1s 33ms/step - loss: 0.6164 - accuracy: 0.7779 - val_loss: 0.6035 - val_accuracy: 0.7779
Epoch 6/40
30/30 [==============================] - 1s 32ms/step - loss: 0.5722 - accuracy: 0.8041 - val_loss: 0.5612 - val_accuracy: 0.7974
Epoch 7/40
30/30 [==============================] - 1s 47ms/step - loss: 0.5229 - accuracy: 0.8290 - val_loss: 0.5159 - val_accuracy: 0.8195
Epoch 8/40
30

In [10]:
results = model.evaluate(test_data, test_labels)
print(results)

782/782 [==============================] - 1s 2ms/step - loss: 0.3348 - accuracy: 0.8705
[0.3347800076007843, 0.8705199956893921]


In [ ]:
Test 데이터들에 적용해본 결과 정확도가 train의 epoch 40 때와 비슷하게 나왔음